In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# 클렌징 1: 불필요한 문자, 기호 등을 사전에 제거 
with open('../data/stevejobs.txt', 'r', encoding='utf8') as f:
    rows = f.readlines()
    lines = [row for row in rows]
text = ' '.join(lines)
print(text[:200])

In [ ]:
# 클렌징 : 정규표현식 사용-불필요한 문자, 기호 등을 사전에 제거 
import re
compile = re.compile("[^ a-zA-Z0-9\.]+")  # ^ 아닌것(부정) 줄바꾸기 제거/ 문장별로 자르지않을거면 . 삭제
text = compile.sub('',text).lower()       # 모두 소문자 처리
print(text[:200])                         # 이후에 .과 숫자 삭제여부 정해야

In [ ]:
# 문장 토큰화 라이브러리 nltk
import nltk         
nltk.download('punkt')                   # nltk의 토큰아이즈 : sent_tokenize  
sentences = nltk.sent_tokenize(text = text)
print(sentences[:3])            # 보통 마침표를 기준으로 자름/ ! ? 줄바꾸기 등으로 자르기도

In [ ]:
# 단어 토큰화
word_token = []
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    word_token.extend(words)
print(word_token[:10])

In [ ]:
# 문서 전체를 넣고 자르기 가능
sentence=nltk.word_tokenize(text)

In [ ]:
# 문서를 문장처럼, 문장을 단어처럼 분석하는 경우도 있음 ?
word_token2 = []
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    word_token2.extend(words)
print(word_token2[:3])

In [ ]:
nltk.word_tokenize('로렘 입숨은 출판이나 그래픽 디자인 분야에서 폰트, 타이포그래피, 레이아웃 같은 그래픽 요소나 시각적 연출을 보여줄 때 사용하는 표준 채우기 텍스트로, 최종 결과물에 들어가는 실제적인 문장 내용이 채워지기 전에 시각 디자인 프로젝트 모형의 채움 글로도 이용된다. 이런 용도로 사용할 때 로렘 입숨을 그리킹(greeking)이라고도 부르며, 때로 로렘 입숨은 공간만 차지하는 무언가를 지칭하는 용어로도 사용된다.')

In [ ]:
# 스톱워드 제거: 분석에 큰 의미없는 단러 제거
nltk.download('stopwords')      # 스톱워드(불용어) 사전
stopwords = nltk.corpus.stopwords.words('english')
print('영어 stop words 개수: ',len(stopwords))
print(stopwords[:10])

In [ ]:
# 텍스트 토큰화 함수
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentense) 
        for sentense in sentences]
    return word_tokens
word_tokens = tokenize_text(text)
print(word_tokens[:2])

In [ ]:
# 스톱워드 제거
all_tokens = []
for sentence in word_tokens:
    filtered_words = []
    for word in sentence:
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
print(all_tokens[:1])

In [ ]:
stopwords.append('.')   #  . 불용어사전에 추가

In [ ]:
# 단어의 원형을 찾는 기법 1) LancasterStemmer -표제어(어근) 추출: 조금 부실
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
print(stemmer.stem('working'))
print(stemmer.stem('works'))
print(stemmer.stem('worked'))

In [ ]:
# 단어의 원형을 찾는 기법 2) WordNetLemmatizer-표제어(어근) 추출: 좀 더 잘 작동
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','a')) 
print(lemma.lemmatize('amuses','a')) 
print(lemma.lemmatize('amused','v'))

print(stemmer.stem('happier'), stemmer.stem('happiest'))
print(lemma.lemmatize('happier','a'))
print(lemma.lemmatize('happiest','a'))

In [ ]:
all_token = []
for i, sentence in enumerate(all_tokens):
    for j, word in enumerate(sentence):
       all_token[i][j]= stemmer.stem(word)
print(all_token[:2])

In [ ]:
# 너무 오래 걸려 안함/ 피처 벡터화: 데이터 가공/ 토큰화/텍스트 정규화/ 피처 벡터화/ 
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
news_data = fetch_20newsgroups(subset = 'all', random_state = 156)
print(news_data.data[0])

In [ ]:
# 이렇게 돌림
from sklearn.datasets import fetch_20newsgroups
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(X_train[0])

In [ ]:
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target
print(f'학습 데이터 크기 {len(train_news.data)}, 테스트 데이터 크기 {len(test_news.data)}')

In [ ]:
## CountVectorizer: 벡터화 할 때 한꺼번에 다 처리
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)
print('학습 데이터 Text의 CountVectorizer Shape:', X_train_cnt_vect.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
data=pd.Series(X_train)
data.head()

In [ ]:
string=''.join(X_train)

In [ ]:
compile = re.compile("[^ a-zA-Z0-9\.]+") 
text=compile.sub('',text).lower()

In [ ]:
words=nltk.word_tokenize(string)
word_series=pd.Series(words)
print(len(word_series))

In [ ]:
X_train_cnt_vect

In [ ]:
cnt_vect.get_feature_names_out()  # 벡터화 후 다시 되돌릴

In [ ]:
train_news.target_names[7]

In [ ]:
cnt_vect.inverse_transform(X_train_cnt_vect.toarray) # ?

In [ ]:
print(X_train_cnt_vect.shape)

In [ ]:
cnt_vect1= CountVectorizer(ngram_range=(1,2), max_features=3000)
cnt_vect1.fit(X_train)
X_train_cnt_vect=cnt_vect1.transform(X_train)
cnt_vect1.get_feature_names_out()[110:200]

In [ ]:
# 테스트용


In [ ]:
cnt_vect= CountVectorizer(max_df=0.85, min_df=0.05, ngram_range=(1,2))
cnt_vect.fit(X_train)
X_train_cnt_vect=cnt_vect.transform(X_train)
X_train_cnt_vect

In [ ]:
cnt_vect.get_feature_names_out()[110:200]

In [ ]:
## TfidfVectorizer: 벡터화 할 때 한꺼번에 다 처리
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

In [ ]:
tfidf_vect = TfidfVectorizer(max_df=0.85,stop_words='english', ngram_range=(1,2), max_df=3000,token_pattern=[a-zA-Z0-9\.])
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_train_tfidf_vect
X_test_tfidf_vect = tfidf_vect.transform(X_test)
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_df=0.85, stop_words='english', max_features=3000, ngram_range=(1,2))
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [ ]:
pd.DataFrame(index=X_train_tfidf_vect,columns=tfidf_vect.get_feature_names_out())   # ?

In [ ]:
compile = re.compile("[^ a-zA-Z0-9\.]+")  

In [ ]:
tfidf_vect.get_feature_names_out()[110:200]

In [ ]:
from sklearn.model_selection import GridSearchCV
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, 
scoring='accuracy', verbose=True )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_ )
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
('lr_clf', LogisticRegression(solver='liblinear'))])
params = {'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
'tfidf_vect__max_df': [100, 300, 700],
'lr_clf__C': [1, 5, 10]}
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
scoring='accuracy', verbose=True)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)
pred = grid_cv_pipe.predict(X_test)
print('예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

#### 한글 텍스트 분석

In [ ]:
with open('../data/소나기.txt', 'r', encoding='utf8') as f:
    text = f.read()
print(text[:200])

In [ ]:
import re
compile = re.compile("[^ ㄱ-ㅣ가-힣\.]+")
text = compile.sub('',text)
print(text[:200])

In [ ]:
# 문장 토큰화
import nltk
sentences = nltk.sent_tokenize(text)
print(sentences[:3])

In [ ]:
okt = Okt()
okt.morphs(text)

In [ ]:
# 단어 토큰화
from konlpy.tag import Okt
okt = Okt()
words = []
for sentence in sentences:
    word = okt.morphs(sentence)
    words.append(word)
print(words[:5])

In [ ]:
nltk.word_tokenize(text)   # 형태소, 조사 제거 안되므로 konlpy

In [164]:
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem=True)
    return tokens_ko
    
word_tokens = okt_tokenizer(text)
print(word_tokens[:20])

NameError: name 'ok_tokenizer' is not defined

In [165]:
# 단어 토큰화
from konlpy.tag import Okt
okt = Okt()
words = []
for sentence in sentences:
    word = okt.morphs(sentence)
    words.append(word)
print(words[:2])

[['소년', '은', '개울가에서', '소녀', '를', '보자', '곧', '윤', '초시', '네', '증손녀', '딸', '이라는', '걸', '알', '수', '있었다', '.'], ['소녀', '는', '개울', '에다', '손', '을', '잠그고', '물장난', '을', '하고', '있는', '것', '이다', '.']]


In [166]:
test_text = '나는 정말로 파이썬을 좋아한다. 아니 머신러닝을 더 좋아한다.'
print('normalize :', okt.normalize(test_text)) # 문장으로 추출
print('morphs :', okt.morphs(test_text))       # 구문 분석
print('nouns :', okt.nouns(test_text))         # 명사만
print('phrases :', okt.phrases(test_text))     # 구문
print('pos :', okt.pos(test_text))             # 품사와 함께 값고 함께 

normalize : 나는 정말로 파이썬을 좋아한다. 아니 머신러닝을 더 좋아한다.
morphs : ['나', '는', '정말로', '파이썬', '을', '좋아한다', '.', '아니', '머신', '러닝', '을', '더', '좋아한다', '.']
nouns : ['나', '파이썬', '머신', '러닝', '더']
phrases : ['파이썬', '머신러닝', '머신', '러닝']
pos : [('나', 'Noun'), ('는', 'Josa'), ('정말로', 'Adverb'), ('파이썬', 'Noun'), ('을', 'Josa'), ('좋아한다', 'Adjective'), ('.', 'Punctuation'), ('아니', 'Adjective'), ('머신', 'Noun'), ('러닝', 'Noun'), ('을', 'Josa'), ('더', 'Noun'), ('좋아한다', 'Adjective'), ('.', 'Punctuation')]
